# Cuaderno 7: Planificación de la producción (2)
# (Varios productos, capacidad de producción)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\ZZ}{\mathbb{Z}_+}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$

Existen numerosas variantes del problema de planificación de la producción (*lot sizing*). Presentamos aquí una variante donde se planifica la producción de dos bienes, tomando en cuenta la capacidad de producción instalada.

Como ejemplo, consideremos la planificación de los niveles óptimos de producción de una empresa dedicada a la producción de dos tipos de barras de cereales, sujeta a las siguientes condiciones:

* El plan de producción se organiza sobre un horizonte de tiempo de un año, dividido en un conjunto $T= \tabulatedset{1, \ldots, 12}$ de períodos mensuales. 
* Si en un período $t \in T$ se toma la decisión de producir (cualquiera de los dos productos), se incurre en un
  costo fijo igual a $c_t$ (en USD). 
* Denotaremos por $I:=\{1,2\}$ al conjunto que representa los dos tipos de barras de cereales a producir. Para cada período $t \in T$ y cada tipo de barra $i \in I$, se conocen la demanda estimada de cada en el mercado $d_{it}$ (en lotes de barras) y el costo unitario de su producción $p_{it}$ (en USD / lote). 
* En cada período se debe satisfacer toda la demanda estimada de cada tipo de barra. Si en un período el nivel de producción supera a la demanda, es posible almacenar el producto excedente a un costo unitario igual a $h_t$ (USD/lote). Este costo es el mismo para ambos tipos de barras. 
* La capacidad de producción de la empresa es constante y está limitada a un total de $C$ lotes por período, entre los dos tipos de barras.
* La producción se realiza por lotes. No es posible producir fracciones de un lote.
  
Se requiere determinar los niveles de producción mensuales para cada tipo de barras, para satisfacer toda la demanda al menor costo posible. Definimos las siguientes variables de decisión:

* variables enteras $x_{it}, \, i \in I, \, t \in T,$ que indican la cantidad de lotes del tipo $i$ a producir en el período $t$;
* variables binarias $y_t, \, t \in T,$ tales que $y_t = 1$ si y solamente si se toma la decisión producir al menos un tipo de barra en el período $t$;
* variables enteras $z_{it}, \, i \in I, \, t \in T,$ que indican la cantidad de lotes de barras del tipo $i$ a almacenar del período $t$ al período $t+1$.

Con estas definiciones, el problema puede ser formulado como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{t \in T} c_t y_t  + \sum_{i \in I} \sum_{t \in T} (p_{it} x_{it} + h_t z_{it})\\ 
& \mbox{s.r.}\\
& x_{i1} = d_{i1} + z_{i1}, \quad \forall i \in I, \\
& x_{it} + z_{i,t-1} = d_{it} + z_{it}, \quad \forall i \in I,  t \in T \setminus \{1\}, \\
& \sum_{i \in I} x_{it} \leq C y_t, \quad \forall t \in T, \\
& y_t \in \{0, 1\}, \quad \forall t \in T, \\
&x_{it}, z_{it} \in \ZZ, \quad \forall i \in I, t \in T.
\end{align*}

La función objetivo mide los costos fijos de producción, los costos variables de producción y los costos de almacenamiento. Los costos fijos de producción y los costos de almacenamiento son independientes del tipo de producto, mientras que los costos variables dependen del producto.

Las dos primeras familias de restricciones establecen, por separado para cada tipo de producto, el balance entre la cantidad a producir, la cantidad almacenada desde el período previo (excepto para el primer período), la demanda y la cantidad a almacenar para el próximo período. Observar que la no negatividad de las variables $z_{it}$ implica que el nivel de producción en cada período debe ser lo suficientemente alto como para satisfacer la demanda de ese período, para cada tipo de producto.

La tercera familia de restricciones contiene restricciones de enforzamiento que indican que $x_{it}$ solamente puede tomar valores mayores a cero, si $y_t = 1$, es decir, si se decide producir en el período $t$. Adicionalmente, en este caso la cantidad total producida en el período $t$ (incluyendo todos los tipos de productos) no debe superar la capacidad de la planta.

Vamos a implementar este programa utilizando la interfaz Python de Gurobi.

Definimos primero los conjuntos $I$ y $T$, y los parámetros $p$, $c$, $h$, $d$ y $C$:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Conjuntos y parámetros del modelo
# Costos fijos y costos de almacenamiento por período:
T, c, h = gp.multidict({
    1 : (400, 10),   2 : (450, 12),  3 : (400, 10),
    4 : (470,  9),   5 : (450, 12),  6 : (400, 10),
    7 : (350, 10),   8 : (400, 12),  9 : (450, 10),
   10 : (450, 12),  11 : (500, 15), 12 : (500,  5)})


# Costos de producción y demanda por (producto-período):
IxT, p, d = gp.multidict({
    (1, 1) : (48, 20),   (1, 2) : (47, 20),  (1, 3) : (38, 25),
    (1, 4) : (45, 25),   (1, 5) : (47, 25),  (1, 6) : (42, 20),
    (1, 7) : (33, 20),   (1, 8) : (39, 18),  (1, 9) : (45, 18),
    (1,10) : (47, 20),   (1,11) : (50, 25),  (1,12) : (55, 27),
    (2, 1) : (44, 15),   (2, 2) : (42, 14),  (2, 3) : (35, 15),
    (2, 4) : (41, 16),   (2, 5) : (42, 15),  (2, 6) : (38, 17),
    (2, 7) : (33, 18),   (2, 8) : (36, 17),  (2, 9) : (41, 15),
    (2,10) : (42, 14),   (2,11) : (45, 15),  (2,12) : (47, 20)})

# Capacidad de la planta (lotes por período)
C = 40

# Recuperar conjunto I de productos desde IxT
I = gp.tuplelist(set(i for (i,t) in IxT))

# Definir conjunto T \ {1}
T2 = [t for t in T if not t==1]

print("T= {}".format(T))
print("I= {}".format(I))
print("IxT= {}".format(IxT))
print("T2= {}".format(T2))
print("c= {}".format(c))
print("h= {}".format(h))
print("p= {}".format(p))
print("d= {}".format(d))
print("C= {}".format(C))

Definimos ahora el objeto modelo y las variables del modelo:

In [ ]:
m = gp.Model('lot-sizing-2')

# lotes a producir
x = m.addVars(I, T, vtype = GRB.INTEGER, name="x")
# alternativa:
# x = m.addVars(IxT, vtype = GRB.INTEGER, name="x")

# se produce / no se produce en cada periodo
y = m.addVars(T, vtype = GRB.BINARY, name="y")

# lotes a almacenar
z = m.addVars(I, T, vtype = GRB.INTEGER, name="z")

Construimos la función objetivo a partir de sus tres términos:

In [ ]:
# costos variables de producción
c_produccion = x.prod(p, '*', '*')

# costos fijos de producción
c_fijo = y.prod(c, '*')

# costos de almacenamiento
almacenamiento = gp.quicksum(h[t]*z[i,t] for i in I for t in T)

m.setObjective(c_produccion + c_fijo + almacenamiento, GRB.MINIMIZE)

Finalmente, implementamos las restricciones del modelo:
1. Balance entre produccion, almacenamiento y demanda.

In [ ]:
# Balance primer período
m.addConstrs((x[i,1] - z[i,1] == d[i,1] for i in I), "balance[1]")

# Balance períodos siguientes
m.addConstrs((x[i,t] + z[i,t-1] - z[i,t] == d[i,t] 
              for i in I for t in T2), "balance")

2. Solamente puede producirse cuando $y_t =1$, y respetando la capacidad de la planta.

In [ ]:
# y[t]=0 => sum_i (x[i, t])=0
m.addConstrs((x.sum('*', t) <= C*y[t] for t in T), "capacidad")

Resolvemos el modelo:

In [ ]:
m.optimize()

Mostramos la solución: producción, demanda y almacenamiento de cada producto en cada período.

In [ ]:
# Extraer valores de las soluciones
vx = m.getAttr('x', x)
vy = m.getAttr('x', y)
vz = m.getAttr('x', z)

# Mostrar tabla con los valores
print('Plan óptimo de producción:')
for i in I:
    print('Producto {}'.format(i))
    print('t\t\tx_t\t\ty_t\t\tz_t\t\td_t')
    for t in T:
        print('{}\t\t{}\t\t{}\t\t{}\t\t{}'.format(t, vx[i,t], vy[t], vz[i,t], d[i,t]))

### Escribir tablas en Python
Existen algunos módulos de Python que facilitan la escritura de tablas en la consola. Uno de ellos es `tabulate`. Para instalar este módulo (si no está disponible aún en el sistema) se usa el comando `pip install`. Luego de ello será necesario **reiniciar el kernel** de este cuaderno.

In [ ]:
# correr esto desde una terminal para instalar tabulate
# pip install --user tabulate

Una vez instalado el módulo, podemos hacer uso de la función `tabulate` dentro del mismo, en combinación con el comando `print`. De esta manera, podemos escribir una lista bidimensional (lista de listas) como una tabla de manera sencilla. La función admite parámetros adicionales para formatear la escritura, uno de ellos es `headers`, que permite especificar los encabezados de las columnas. Otros detalles pueden ser consultados en [esta página web.](https://pypi.org/project/tabulate/)

In [ ]:
from tabulate import tabulate

# Extraer valores de las soluciones
vx = m.getAttr('x', x)
vy = m.getAttr('x', y)
vz = m.getAttr('x', z)

# Mostrar tabla con los valores
for i in I:
    print('Producto {}'.format(i))
    tabla = [[t, vx[i,t], vy[t], vz[i,t], d[i,t]] for t in T]
    # print(tabla)
    # print(tabulate(tabla))
    print(tabulate(tabla, headers=['t', 'x_it', 'y_t', 'z_it', 'd_it']))

## Código completo

Se reproduce a continuación el código completo del modelo anterior. En esta versión es posible modificar el número de productos.

In [ ]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de planificación de la producción con varios productos y capacidad limitada de producción
# EPN (2023)

import gurobipy as gp
from gurobipy import GRB
from random import randint, seed

try:
    # Número de productos
    n = 10
    
    # Conjuntos y parametros del modelo
    # Costos fijos y costos de almacenamiento por período:
    T, c, h = gp.multidict({
        1 : (200*n, 10),   2 : (270*n, 12),  3 : (200*n, 10),
        4 : (250*n,  9),   5 : (270*n, 12),  6 : (200*n, 10),
        7 : (180*n, 10),   8 : (200*n, 12),  9 : (225*n, 10),
       10 : (270*n, 12),  11 : (250*n, 15), 12 : (250*n,  5)})

    # Conjunto de productos
    I = gp.tuplelist(i+1 for i in range(n))
    
    # Costos de producción por (producto-período):
    seed(0)
    p = gp.tupledict({(i,t) : randint(40,60) for i in I for t in T})
    
    # Demandas estimadas por (producto-período):
    d = gp.tupledict({(i,t) : randint(15,25) for i in I for t in T})

    # Capacidad de la planta (lotes por período)
    C = 80*n

    # Definir conjunto T \ {1}
    T2 = [t for t in T if not t==1]
    
    # Crear el objeto modelo
    m = gp.Model('lot-sizing-2')

    # Variables de decisión
    # lotes a producir
    x = m.addVars(I, T, vtype = GRB.INTEGER, name="x")

    # se produce / no se produce en cada período
    y = m.addVars(T, vtype = GRB.BINARY, name="y")

    # lotes a almacenar
    z = m.addVars(I, T, vtype = GRB.INTEGER, name="z")    

    # Función objetivo
    # costos variables de producción
    c_produccion = x.prod(p, '*')

    # costos fijos de producción
    c_fijo = y.prod(c, '*')

    # costos de almacenamiento
    almacenamiento = gp.quicksum([h[t]*z[i,t] for i in I for t in T])

    m.setObjective(c_produccion + c_fijo + almacenamiento, GRB.MINIMIZE)

    # Restricciones
    # balance primer período
    m.addConstrs((x[i,1] - z[i,1] == d[i,1] for i in I), "balance_ini")

    # balance períodos siguientes
    m.addConstrs((x[i,t] + z[i,t-1] - z[i,t] == d[i,t] for i in I for t in T2), "balance")   
    
    # y[t]=0 => sum_i (x[i, t])=0
    m.addConstrs((x.sum('*', t) <= C*y[t] for t in T), "capacidad")   
    
    # Resolver el modelo
    m.optimize()
    
    # Mostrar solución
    # extraer valores de las soluciones
    vx = m.getAttr('x', x)
    vy = m.getAttr('x', y)
    vz = m.getAttr('x', z)

    # mostrar tabla con los valores
    print('Plan óptimo de producción:')
    for i in I:
        print('Producto {}'.format(i))
        print('t\t\tx_t\t\ty_t\t\tz_t\t\td_t')
        for t in T:
            print('{}\t\t{}\t\t{}\t\t{}\t\t{}'.format(t, vx[i,t], vy[t], vz[i,t], d[i,t]))  
            
except GurobiError as e:
    print('Se produjo un error de Gurobi: código: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')